In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


%run nb_init.py
# This imports leg_joint as lj


/home/guillaume/anaconda/envs/python3/lib/python3.4/site-packages/graph_tool/draw/cairo_draw.py:1340: RuntimeWarning: Error importing Gtk module: No module named 'gi'; GTK+ drawing will not work.
  warnings.warn(msg, RuntimeWarning)
2015-08-23 19:32:25,279 -leg_joint -INFO -successfully imported leg_joint


In [2]:
lj_eptm = lj.Epithelium(lj.data.before_apoptosis_xml(), save_dir='/tmp')

2015-08-23 19:32:26,484 -leg_joint.epithelium -INFO -Instanciating epithelium 0
2015-08-23 19:32:27,756 -leg_joint.epithelium -INFO -Initial cells
2015-08-23 19:32:27,757 -leg_joint.epithelium -INFO -Initial junctions
2015-08-23 19:32:34,010 -leg_joint.epithelium -INFO -Update geometry


In [10]:
import numpy as np
import pandas as pd

import matplotlib.pylab as plt
%matplotlib inline

import tyssue as ty
from tyssue.core import Epithelium, Cell, JunctionEdge, JunctionVertex
from tyssue.core.generation import jv_data, cell_data, je_data
from tyssue.geometry import sheet_geometry as geom

In [18]:
graph = lj_eptm.graph
print('****** Vertex ********')
print(*graph.vertex_properties.keys(), sep='\n')
print('****** Edge ********')

print(*graph.edge_properties.keys(), sep='\n')

      

****** Vertex ********
volume_grad
grad_zed
grad_radial
volume_grad_cell
vol_elasticities
volume_grad_radial
grad_sigma
contractile_grad
prefered_vol
volume_grad_apical
wys
is_active_vert
zeds
areas
ixs
grad_ix
is_local_vert
vols
perimeters
rhos
ages
is_alive
radial_tensions
grad_wy
sigmas
elastic_grad
contractilities
thetas
is_cell_vert
****** Edge ********
u_drhos
dixs
dthetas
drhos
dwys
u_dixs
is_active_edge
dzeds
u_dzeds
u_dwys
u_dsigmas
edge_lengths
is_ctoj_edge
line_tensions
is_local_edge
at_boundary
is_new_edge
dsigmas
is_junction_edge


In [14]:
print('CELLS')
print(*cell_data.keys(), sep='\t')

print('JV')
print(*jv_data.keys(), sep='\t')

print('JE')
print(*je_data.keys(), sep='\t')

CELLS
z	x	perimeter	y	area	is_alive	num_sides
JV
z	is_active	x	y
JE
nz	length	dx	dz	dy	ny	nx


In [19]:
jv_import = {'ixs': 'x', 'wys': 'y', 'zeds': 'z',
             'is_active_vert': 'is_active'}

cell_import = {'ixs': 'x', 'wys': 'y', 'zeds': 'z',
               'area': 'area', 'perimeter': 'perimeter',
               'is_alive': 'is_alive'}

je_import = {'dixs': 'dx', 'dwys': 'dy', 'dzeds': 'dz',
             'u_dixs': 'dx', 'u_dwys': 'dy', 'u_dzeds': 'nz',
             'edge_lengths': 'length'}

In [20]:
def get_faces(graph, as_array=True):
    '''
    Retrieves all the triangular subgraphs of the form
       1 -- > 2
        ^   ^
         \ /
          0
    In our context, vertex 0 always corresponds to a cell
    and vertices 1 and 2 to junction vertices

    Parameters
    ----------
    graph : a :class:`GraphTool` graph instance
    as_array: bool, optional, default `True`
      if `True`, the output of `subraph_isomorphism` is converted
      to a (N, 3) ndarray.

    Returns
    -------
    triangles:  list of gt.PropertyMaps or (N, 3) ndarray
      each line corresponds to a triplet (cell, jv0, jv1)
      where cell, jv0 and jv1 are indices of the input graph.
    '''
    tri_graph = gt.Graph()
    ## the vertices
    verts = tri_graph.add_vertex(3)
    ## edges
    tri_graph.add_edge_list([(0, 1), (0, 2), (1, 2)])
    _triangles = gt.subgraph_isomorphism(tri_graph, graph)
    if not as_array:
        return tri_graph, _triangles
    triangles = np.array([tri.a for tri in _triangles], dtype=np.int)
    return triangles

In [21]:
triangles = get_faces(graph)

The new base triangle is oriented differently, as the edge is splited.

We now have:

```
       1 -- > 2             2 <-- 1  
        ^   /                ^   /
         \ v     and          \ v  
          0                    0
```



In [ ]:
graph.set_vertex_filter(lj_eptm.is_cell_vert)
graph.vertex_index

graph.set_vertex_filter(lj_eptm.is_cell_vert)